# Reading the Data

In [1]:
import pandas as pd
df_matched = pd.read_excel("del7_ADRD_with_map_3new.xlsx")

In [5]:
print(df_dropped.columns.tolist())


['study_id', 'study_enc_id', 'study_case_id', 'duration', 'del_1', 'year-proc', 'sex', 'race', 'ethnicity', 'age', 'asa_class', 'asa1', 'asa1e', 'asa2', 'asa2e', 'asa3', 'asa3e', 'asa4', 'asa4e', 'asa5', 'asa5e', 'asa6', 'asa6e', 'CCI', 'cisatracurium', 'hydromorphone', 'lorazepam', 'midazolam', 'sufentanil', 'alfentanil', 'dexmedetomidine', 'etomidate', 'fentanyl', 'ketamine', 'methadone', 'meperidine', 'morphine', 'propofol', 'remifentanil', 'rocuronium', 'succinylcholine', 'vecuronium', 'diazepam', 'map_min', 'map_max', 'esmolol', 'hydralazine', 'labetalol', 'metoprolol', 'nicardipine', 'nitroprusside', 'enalapril', 'epinephrine', 'norepinephrine', 'phenylephrine', 'ephedrine', 'vasopressin', 'dopamine', 'milrinone', 'dobutamine', 'SE', 'Anemia', 'CKD', 'COPD', 'Cancer', 'Cancer METS', 'Cardiac arrhythmia', 'Cerebral Vascular Dz', 'Chr Dialysis', 'Chr ETOH', 'Chr heart failure', 'Diabetes', 'Disease of the AORTA', 'Drug Abuse', 'HTN', 'Heart valve dz', 'Hemiplegia/Paraplegia', 'Isch

# Exploratory Data Analysis

In [3]:
df_matched[df_matched["outcome"] != df_matched["del_1"]]

,Unnamed: 0,case_x,study_id,study_enc_id,study_case_id,duration,outcome,del_1,del_8,year-proc,...,GU,ENT,EP/PACEMAKER,GI-NORA,EYE,case_y,Dementia,map_lt_65_min,map_65_95_min,map_gt_95_min


In [3]:
suspects = ['anti-parkinsonian', 'antidepressants', 'Dementia', 'Muscle Relaxants', 'Antimuscarinics/Incontinence Meds']
print("suspects unique values:")
for col in suspects:
    print(f"{col}: {df_matched[col].unique()}")

suspects unique values:
anti-parkinsonian: [0]
antidepressants: [0]
Dementia: [0 1]
Muscle Relaxants: [0 1]
Antimuscarinics/Incontinence Meds: [0]


In [4]:
df_matched['ethnicity'].value_counts()

ethnicity
Not Hispanic or Latino    5562
Hispanic or Latino          98
Unknown                     52
Declined                     6
Name: count, dtype: int64

# Data Preprocessing

In [ ]:
df_dropped = df_matched.drop(columns=['map_mean', 'outcome','Case','Muscle Relaxants', 'case_x', 'case_y', 'del_8', 'Unnamed: 0', 'case', 'Antimuscarinics/Incontinence Meds', 'asa_class.1', 'age.1', 'anti-parkinsonian', 'antidepressants', 'Respiratory Medications', 'study_id_case', 'study_enc_id_case', 'study_case_id_case'], errors="ignore")
df_dropped

In [3]:
pd.set_option('display.max_columns', None)

## Medication Class Binarization (Presence/Absence Encoding)

In [4]:
mask_cols = [
    'anti histaminics', 'Antispasmotics', 'sedatives',
    'antihypertensives', 'Gastrointestinal Agents',
    'opioids and NSAIDS', 'psychoactive and anticonvulsant ',
    'steroids', 'anti-arrhythmics', 'antibiotics'
]

# Keep originals with suffix "_orig"
for col in mask_cols:
    df_dropped[col + "_orig"] = df_dropped[col]
    df_dropped[col] = (df_dropped[col] > 0).astype(int)   

In [5]:
# Clean column names (strip, lower, replace spaces)
df_dropped.columns = (
    df_dropped.columns
      .str.strip()
      .str.replace(r'\s+', ' ', regex=True)
)

## Feature Engineering for ASA Class and Emergency Status

In [6]:
# --- Step 1: Drop "_orig" columns ---
df_mod = df_dropped.loc[:, ~df_dropped.columns.str.endswith("_orig")].copy()
print(f"Dropped _orig columns. Remaining columns: {df_mod.shape[1]}")

# --- Step 2: Create ASA numeric and emergency flag ---
# ASA class numeric: extract from asa_class (if present)
if "asa_class" in df_mod.columns:
    # Coerce text to numeric safely
    df_mod["asa_class_num"] = (
        df_mod["asa_class"]
        .astype(str)
        .str.extract(r"(\d)")
        .astype(float)
    )

# Emergency flag: 1 if any of the ASA_emergency variants (asa1e–asa6e) are 1
asa_emerg_cols = [c for c in df_mod.columns if c.lower().startswith("asa") and c.lower().endswith("e")]
df_mod["asa_emerg_flag"] = df_mod[asa_emerg_cols].max(axis=1) if asa_emerg_cols else 0

# --- Step 3: Drop individual ASA one-hot columns (asa1–asa6 and asa1e–asa6e) ---
asa_cols_to_drop = [c for c in df_mod.columns if c.lower().startswith("asa") and (c.lower().endswith(tuple(str(i) for i in range(1, 7))) or c.lower().endswith(tuple(f"{i}e" for i in range(1, 7))))]
df_mod.drop(columns=asa_cols_to_drop, inplace=True, errors="ignore")

print(f"Created asa_class_num + asa_emerg_flag and dropped {len(asa_cols_to_drop)} ASA dummies.")
print(df_mod[["asa_class", "asa_class_num", "asa_emerg_flag"]].head())


Dropped _orig columns. Remaining columns: 113
Created asa_class_num + asa_emerg_flag and dropped 12 ASA dummies.
  asa_class  asa_class_num  asa_emerg_flag
0         3            3.0               0
1         3            3.0               0
2         3            3.0               0
3         3            3.0               0
4         4            4.0               0


In [7]:
print(df_mod[["asa_class", "asa_class_num", "asa_emerg_flag"]].head(20))

   asa_class  asa_class_num  asa_emerg_flag
0          3            3.0               0
1          3            3.0               0
2          3            3.0               0
3          3            3.0               0
4          4            4.0               0
5          3            3.0               0
6          4            4.0               0
7          4            4.0               0
8         3E            3.0               1
9         4E            4.0               1
10         3            3.0               0
11        4E            4.0               1
12        4E            4.0               1
13        5E            5.0               1
14        5E            5.0               1
15        4E            4.0               1
16         3            3.0               0
17        4E            4.0               1
18         3            3.0               0
19         3            3.0               0


## Phik-Based Feature Correlation Screening

In [8]:
# --- 1. Drop redundant column ---
df_phik = df_mod.drop(columns=["asa_class"], errors="ignore").copy()

# --- 2. Import and compute Phik correlation matrix ---
from phik import phik_matrix
from phik.report import plot_correlation_matrix
import numpy as np, pandas as pd

# Numeric columns (for better binning)
interval_cols = df_phik.select_dtypes(include=["number"]).columns.tolist()

# Compute Phik correlation matrix
phik_mat = phik_matrix(df_phik, interval_cols=interval_cols)

# --- 3. Flatten into long-form pairs ---
np.fill_diagonal(phik_mat.values, np.nan)
mask_upper = np.triu(np.ones_like(phik_mat, dtype=bool), k=1)
pairs = (
    phik_mat.where(mask_upper)
            .stack()
            .reset_index()
            .rename(columns={"level_0": "col1", "level_1": "col2", 0: "phik"})
            .sort_values("phik", ascending=False)
)

# --- 4. View highly correlated pairs (>|0.8|) ---
high_pairs = pairs.loc[pairs["phik"] > 0.8].reset_index(drop=True)
print("Highly correlated pairs (>|0.8|):")
print(high_pairs.head(50).to_string(index=False))

# Optional visualization (good for overview)
# plot_correlation_matrix(phik_mat.values, x_labels=phik_mat.columns, y_labels=phik_mat.index, vmin=0.5, vmax=1.0)


Highly correlated pairs (>|0.8|):
         col1                            col2     phik
 study_enc_id                   study_case_id 0.999921
     ketamine psychoactive and anticonvulsant 0.998867
    midazolam                       sedatives 0.996790
study_case_id                       year-proc 0.990390
 study_enc_id                       year-proc 0.990062
     fentanyl              opioids and NSAIDS 0.975181
          CKD                    Chr Dialysis 0.971636
   metoprolol               antihypertensives 0.948791
     Chr ETOH                      Drug Abuse 0.839514


In [9]:
# --- Drop redundant high-correlation features (based on expert review) ---
drop_cols = [
    "psychoactive and anticonvulsant",
    "sedatives",
    "Chr Dialysis",
    "opioids and NSAIDS",
    "antihypertensives",
    "Drug Abuse"
]

df_mod = df_mod.drop(columns=drop_cols, errors="ignore")

print(f"Dropped {len(drop_cols)} highly correlated columns:")
print(drop_cols)
print(f"\nRemaining columns: {df_mod.shape[1]}")


Dropped 6 highly correlated columns:
['psychoactive and anticonvulsant', 'sedatives', 'Chr Dialysis', 'opioids and NSAIDS', 'antihypertensives', 'Drug Abuse']

Remaining columns: 97


In [15]:
unique_names = df_mod['ethnicity'].unique()
print(unique_names)

['Not Hispanic or Latino' 'Unknown' nan 'Hispanic or Latino' 'Declined']


In [19]:
print(df_mod.columns.tolist())


['study_id', 'study_enc_id', 'study_case_id', 'duration', 'del_1', 'year-proc', 'sex', 'race', 'ethnicity', 'age', 'asa_class', 'CCI', 'cisatracurium', 'hydromorphone', 'lorazepam', 'midazolam', 'sufentanil', 'alfentanil', 'dexmedetomidine', 'etomidate', 'fentanyl', 'ketamine', 'methadone', 'meperidine', 'morphine', 'propofol', 'remifentanil', 'rocuronium', 'succinylcholine', 'vecuronium', 'diazepam', 'map_min', 'map_max', 'esmolol', 'hydralazine', 'labetalol', 'metoprolol', 'nicardipine', 'nitroprusside', 'enalapril', 'epinephrine', 'norepinephrine', 'phenylephrine', 'ephedrine', 'vasopressin', 'dopamine', 'milrinone', 'dobutamine', 'SE', 'Anemia', 'CKD', 'COPD', 'Cancer', 'Cancer METS', 'Cardiac arrhythmia', 'Cerebral Vascular Dz', 'Chr ETOH', 'Chr heart failure', 'Diabetes', 'Disease of the AORTA', 'HTN', 'Heart valve dz', 'Hemiplegia/Paraplegia', 'Ischemic heart dz', 'LIVER DZ', 'Obesity', 'Peripheral vascular disease', 'Psychiatric Disorders', 'Pulmonaru Vasular Dz', 'Resp Failure

## Missing values check and imputation

In [10]:
# --- Missing value summary for df_dropped ---
import pandas as pd

missing_summary = (
    df_mod.isna()
    .sum()
    .to_frame("n_missing")
    .assign(pct_missing=lambda d: 100 * d["n_missing"] / len(df_mod))
    .query("n_missing > 0")  # only columns with missing values
    .sort_values("pct_missing", ascending=False)
)

print("=== Missing value summary (only columns with >0 missing) ===")
display(missing_summary.head(30))  # show top 30 for quick scan
print(f"\nTotal columns with missing values: {missing_summary.shape[0]}")
print(f"Total rows: {len(df_mod)}")


=== Missing value summary (only columns with >0 missing) ===


,n_missing,pct_missing
ethnicity,30,0.523652
asa_class,1,0.017455
asa_class_num,1,0.017455



Total columns with missing values: 3
Total rows: 5729


In [11]:
# --- Handle missing values in df_mod ---
df_mod = df_mod.copy()

# 1) Fill missing ethnicity with 'Unknown'
df_mod["ethnicity"] = df_mod["ethnicity"].fillna("Unknown")

# 2) Impute ASA class and ASA class_num
if "asa_class" in df_mod.columns:
    mode_asa = df_mod["asa_class"].mode(dropna=True)[0]
    df_mod["asa_class"] = df_mod["asa_class"].fillna(mode_asa)

if "asa_class_num" in df_mod.columns:
    median_asa_num = df_mod["asa_class_num"].median()
    df_mod["asa_class_num"] = df_mod["asa_class_num"].fillna(median_asa_num)

# --- Quick verification ---
print("Remaining missing values after imputation:")
print(df_mod.isna().sum().loc[lambda x: x > 0])


Remaining missing values after imputation:
Series([], dtype: int64)


# Feature Schema Definition for Reproducible Modeling

In [12]:
# --- Build & save feature schema from df_mod (final) ---
import json, numpy as np, pandas as pd
from pathlib import Path

TARGET = "del_1"

# Exclude IDs & admin columns from features, but keep them in df_mod for CV grouping
ID_COLS = ["study_id", "study_enc_id", "study_case_id"]
ADMIN_COLS = ["year-proc", "asa_class"]  # asa_class_num stays as continuous; raw asa_class is redundant
EXCLUDE = [TARGET] + ID_COLS + ADMIN_COLS

df = df_mod.copy()

# 1) Constants (<=1 unique non-null)
constant_cols = (
    df.drop(columns=EXCLUDE, errors="ignore")
      .nunique(dropna=True)
      .pipe(lambda s: s[s <= 1].index.tolist())
)

# 2) Split by dtype
num_like = df.select_dtypes(include=[np.number, "boolean", "bool"]) \
             .drop(columns=EXCLUDE + constant_cols, errors="ignore")
obj_like = df.select_dtypes(include=["object", "category"]) \
             .drop(columns=EXCLUDE + constant_cols, errors="ignore")

# 3) Detect strict 0/1 binaries among numerics
def is_binary(series: pd.Series) -> bool:
    vals = pd.Series(series).dropna().unique()
    return len(vals) > 0 and set(vals).issubset({0, 1})

binary_cols = [c for c in num_like.columns if is_binary(num_like[c])]
continuous_cols = sorted(set(num_like.columns) - set(binary_cols))
categorical_cols = sorted(obj_like.columns.tolist())

# 4) Enforce ASA design explicitly (keep numeric; flag emergency as binary)
if "asa_class_num" in df.columns:
    if "asa_class_num" in binary_cols:   binary_cols.remove("asa_class_num")
    if "asa_class_num" not in continuous_cols and "asa_class_num" not in constant_cols:
        continuous_cols.append("asa_class_num")

if "asa_emerg_flag" in df.columns and is_binary(df["asa_emerg_flag"]):
    if "asa_emerg_flag" in continuous_cols: continuous_cols.remove("asa_emerg_flag")
    if "asa_emerg_flag" not in binary_cols and "asa_emerg_flag" not in constant_cols:
        binary_cols.append("asa_emerg_flag")

# 5) Final feature list (exclude target, IDs, admin, constants)
feature_list = [c for c in df.columns if c not in set(EXCLUDE + constant_cols)]

# Ensure mutual exclusivity across buckets (priority: categorical > binary > continuous)
seen = set(); final_cats, final_bins, final_conts = [], [], []
for c in sorted(categorical_cols):
    if c in feature_list and c not in seen: final_cats.append(c); seen.add(c)
for c in sorted(binary_cols):
    if c in feature_list and c not in seen: final_bins.append(c); seen.add(c)
for c in sorted(continuous_cols):
    if c in feature_list and c not in seen: final_conts.append(c); seen.add(c)

# Ordered feature list (optional; your choice)
feature_list = final_conts + final_cats + final_bins

# 6) Label stats
label_stats = {TARGET: float(pd.Series(df[TARGET]).mean())}

# 7) Save schema
schema = {
    "target": TARGET,
    "aux_outcomes": [],
    "binary_cols": sorted(final_bins),
    "continuous_cols": sorted(final_conts),
    "categorical_cols": sorted(final_cats),
    "constant_cols": sorted(constant_cols),
    "feature_list": feature_list,
    "dtypes": {c: str(df[c].dtype) for c in feature_list},
    "label_stats": label_stats,
    "exclude_cols": EXCLUDE,        # helpful breadcrumb
    "id_cols": ID_COLS,             # for grouping later
}

Path("artifacts").mkdir(exist_ok=True)
with open("artifacts/feature_schema.json", "w") as f:
    json.dump(schema, f, indent=2)

print("Saved → artifacts/feature_schema.json")
print(f"Counts → continuous={len(schema['continuous_cols'])}, categorical={len(schema['categorical_cols'])}, binary={len(schema['binary_cols'])}, constants={len(schema['constant_cols'])}")
print("Categoricals:", schema["categorical_cols"])
print("Target prevalence:", f"{schema['label_stats'][TARGET]:.3f}")
print("Excluded (not modeled):", EXCLUDE)


Saved → artifacts/feature_schema.json
Counts → continuous=10, categorical=3, binary=78, constants=0
Categoricals: ['ethnicity', 'race', 'sex']
Target prevalence: 0.500
Excluded (not modeled): ['del_1', 'study_id', 'study_enc_id', 'study_case_id', 'year-proc', 'asa_class']


In [14]:
continuous_cols

['CCI',
 'age',
 'asa_class_num',
 'duration',
 'map_65_95_min',
 'map_gt_95_min',
 'map_lt_65_min',
 'map_max',
 'map_min',
 'total_mac_hrs']

In [20]:
binary_cols

['cisatracurium',
 'hydromorphone',
 'lorazepam',
 'midazolam',
 'sufentanil',
 'alfentanil',
 'dexmedetomidine',
 'etomidate',
 'fentanyl',
 'ketamine',
 'methadone',
 'meperidine',
 'morphine',
 'propofol',
 'remifentanil',
 'rocuronium',
 'succinylcholine',
 'vecuronium',
 'diazepam',
 'esmolol',
 'hydralazine',
 'labetalol',
 'metoprolol',
 'nicardipine',
 'nitroprusside',
 'enalapril',
 'epinephrine',
 'norepinephrine',
 'phenylephrine',
 'ephedrine',
 'vasopressin',
 'dopamine',
 'milrinone',
 'dobutamine',
 'SE',
 'Anemia',
 'CKD',
 'COPD',
 'Cancer',
 'Cancer METS',
 'Cardiac arrhythmia',
 'Cerebral Vascular Dz',
 'Chr ETOH',
 'Chr heart failure',
 'Diabetes',
 'Disease of the AORTA',
 'HTN',
 'Heart valve dz',
 'Hemiplegia/Paraplegia',
 'Ischemic heart dz',
 'LIVER DZ',
 'Obesity',
 'Peripheral vascular disease',
 'Psychiatric Disorders',
 'Pulmonaru Vasular Dz',
 'Resp Failure',
 'Transplanted Organ',
 'anti histaminics',
 'Antispasmotics',
 'Gastrointestinal Agents',
 'stero

In [21]:
categorical_cols

['ethnicity', 'race', 'sex']

# Unfitted Feature Preprocessing for Cross-Validated Modeling

In [13]:
# A) Build & save UNFITTED preprocessor (no leakage, no model)
import json
import joblib
from pathlib import Path
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.impute import SimpleImputer

# 0) Load schema produced earlier
with open("artifacts/feature_schema.json", "r") as f:
    schema = json.load(f)

TARGET            = schema["target"]
continuous_cols   = schema["continuous_cols"]
categorical_cols  = schema["categorical_cols"]   # <-- fixed spelling
binary_cols       = schema["binary_cols"]
EXCLUDED          = schema.get("exclude_cols", [])
ID_COLS           = schema.get("id_cols", ["study_id", "study_enc_id", "study_case_id"])

# 1) Resolve columns actually present in df_mod (guard against drift)
present_cont = [c for c in continuous_cols  if c in df_mod.columns]
present_cat  = [c for c in categorical_cols if c in df_mod.columns]
present_bin  = [c for c in binary_cols      if c in df_mod.columns]

# Safety: ensure excluded + target are not in any present_* lists
ban = set(EXCLUDED + [TARGET])
present_cont = [c for c in present_cont if c not in ban]
present_cat  = [c for c in present_cat  if c not in ban]
present_bin  = [c for c in present_bin  if c not in ban]

# 2) Define unfitted transformers
try:
    # Newer sklearn (>=1.2)
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=True)
except TypeError:
    # Older sklearn
    ohe = OneHotEncoder(handle_unknown="ignore", sparse=True)

cont_pipe = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("scale",  RobustScaler()),
])

cat_pipe = Pipeline([
    ("impute", SimpleImputer(strategy="most_frequent")),
    ("ohe",    ohe),
])

bin_pipe = Pipeline([
    ("impute", SimpleImputer(strategy="most_frequent")),
])

# 3) ColumnTransformer (UNFITTED) — no fit/transform here
preprocessor = ColumnTransformer(
    transformers=[
        ("cont", cont_pipe, present_cont),
        ("cat",  cat_pipe,  present_cat),
        ("bin",  bin_pipe,  present_bin),
    ],
    remainder="drop",
    verbose_feature_names_out=False
)

# 4) Save the unfitted preprocessor and a spec for reproducibility
Path("artifacts").mkdir(exist_ok=True)
joblib.dump(preprocessor, "artifacts/preprocessor_unfitted.joblib")

spec = {
    "target": TARGET,
    "excluded": EXCLUDED,
    "id_cols": ID_COLS,
    "present_cont": present_cont,
    "present_cat": present_cat,
    "present_bin": present_bin,
}
with open("artifacts/preprocessor_spec.json", "w") as f:
    json.dump(spec, f, indent=2)

print("Saved → artifacts/preprocessor_unfitted.joblib")
print("Spec   → artifacts/preprocessor_spec.json")
print(f"Blocks → continuous={len(present_cont)}, categorical={len(present_cat)}, binary={len(present_bin)}")
print("Not modeled (kept only for grouping/metadata):", EXCLUDED)


Saved → artifacts/preprocessor_unfitted.joblib
Spec   → artifacts/preprocessor_spec.json
Blocks → continuous=10, categorical=3, binary=78
Not modeled (kept only for grouping/metadata): ['del_1', 'study_id', 'study_enc_id', 'study_case_id', 'year-proc', 'asa_class']


# Baseline Models

## Baseline Logistic Regression with StratifiedGroupKFold

In [16]:
# B) Baseline Logistic Regression with StratifiedGroupKFold (group = STUDY_ID)

import json
import numpy as np
import pandas as pd
import joblib
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss

# 0) Load artifacts from Step A
preprocessor = joblib.load("artifacts/preprocessor_unfitted.joblib")
with open("artifacts/preprocessor_spec.json", "r") as f:
    spec = json.load(f)

TARGET   = spec["target"]
EXCLUDED = spec["excluded"]

# 1) Build X/y (exclude target + excluded cols); define patient groups
drop_cols = list(set(EXCLUDED + [TARGET]))
X_all = df_mod.drop(columns=drop_cols).copy()
y_all = df_mod[TARGET].astype(int).copy()

if "study_id" not in df_mod.columns:
    raise ValueError("study_id column is required for patient-level grouped CV.")
groups = df_mod["study_id"].values

# 2) Baseline Logistic Regression (L2)
lr = LogisticRegression(
    penalty="l2",
    solver="liblinear",   # robust with sparse OHE
    C=1.0,
    max_iter=5000,
    random_state=42
)

pipe_lr = Pipeline([
    ("prep", preprocessor),  # unfitted; fits within each fold
    ("clf",  lr),
])

# 3) StratifiedGroupKFold by patient
n_splits = 5
cv = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state=42)
split_iter = cv.split(X_all, y_all, groups=groups)

# 4) Train/evaluate per fold; collect OOF predictions
oof_pred = np.zeros(len(y_all), dtype=float)
fold_metrics = []

for fold, (tr_idx, va_idx) in enumerate(split_iter, start=1):
    X_tr, X_va = X_all.iloc[tr_idx], X_all.iloc[va_idx]
    y_tr, y_va = y_all.iloc[tr_idx], y_all.iloc[va_idx]

    model = pipe_lr
    model.fit(X_tr, y_tr)

    p = model.predict_proba(X_va)[:, 1]
    oof_pred[va_idx] = p

    auc  = roc_auc_score(y_va, p)
    pr   = average_precision_score(y_va, p)
    bri  = brier_score_loss(y_va, p)
    fold_metrics.append((auc, pr, bri))
    print(f"[Fold {fold}] ROC-AUC={auc:.3f}  PR-AUC={pr:.3f}  Brier={bri:.3f}")

# 5) Overall OOF performance (primary CV estimate)
auc_oof = roc_auc_score(y_all, oof_pred)
pr_oof  = average_precision_score(y_all, oof_pred)
bri_oof = brier_score_loss(y_all, oof_pred)

print("\n=== Cross-validated (OOF) performance ===")
print(f"ROC-AUC={auc_oof:.3f}  PR-AUC={pr_oof:.3f}  Brier={bri_oof:.3f}")

# 6) (Optional) Per-fold mean ± SD
fold_metrics = np.array(fold_metrics)
print("\nPer-fold means ± SD:")
print(f"AUC  : {fold_metrics[:,0].mean():.3f} ± {fold_metrics[:,0].std():.3f}")
print(f"PR   : {fold_metrics[:,1].mean():.3f} ± {fold_metrics[:,1].std():.3f}")
print(f"Brier: {fold_metrics[:,2].mean():.3f} ± {fold_metrics[:,2].std():.3f}")


[Fold 1] ROC-AUC=0.831  PR-AUC=0.807  Brier=0.168
[Fold 2] ROC-AUC=0.818  PR-AUC=0.811  Brier=0.175
[Fold 3] ROC-AUC=0.803  PR-AUC=0.790  Brier=0.181
[Fold 4] ROC-AUC=0.831  PR-AUC=0.831  Brier=0.168
[Fold 5] ROC-AUC=0.798  PR-AUC=0.778  Brier=0.184

=== Cross-validated (OOF) performance ===
ROC-AUC=0.816  PR-AUC=0.802  Brier=0.175

Per-fold means ± SD:
AUC  : 0.816 ± 0.014
PR   : 0.803 ± 0.018
Brier: 0.175 ± 0.007


# Baseline Suite: LR-ElasticNet, RandomForest, HistGradientBoosting

In [14]:
# Baseline Suite: LR-ElasticNet, RandomForest, HistGradientBoosting
# (StratifiedGroupKFold, group = STUDY_ID; no domain grouping)

import json, joblib, numpy as np, pandas as pd
from pathlib import Path
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_hist_gradient_boosting  # noqa: F401
from sklearn.ensemble import HistGradientBoostingClassifier

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.impute import SimpleImputer

# ---------- Load artifacts/spec ----------
preprocessor_sparse = joblib.load("artifacts/preprocessor_unfitted.joblib")
with open("artifacts/preprocessor_spec.json", "r") as f:
    spec = json.load(f)

TARGET   = spec["target"]
EXCLUDED = spec["excluded"]

# ---------- Build X/y and groups ----------
drop_cols = list(set(EXCLUDED + [TARGET]))
X_all = df_mod.drop(columns=drop_cols).copy()
y_all = df_mod[TARGET].astype(int).copy()
groups = df_mod["study_id"].values

# ---------- Helper: dense preprocessor for HGB ----------
present_cont = spec["present_cont"]
present_cat  = spec["present_cat"]
present_bin  = spec["present_bin"]

def make_dense_preprocessor():
    try:
        ohe_dense = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    except TypeError:
        ohe_dense = OneHotEncoder(handle_unknown="ignore", sparse=False)

    cont_pipe = Pipeline([
        ("impute", SimpleImputer(strategy="median")),
        ("scale",  RobustScaler()),
    ])
    cat_pipe = Pipeline([
        ("impute", SimpleImputer(strategy="most_frequent")),
        ("ohe",    ohe_dense),
    ])
    bin_pipe = Pipeline([
        ("impute", SimpleImputer(strategy="most_frequent")),
    ])

    return ColumnTransformer(
        transformers=[
            ("cont", cont_pipe, present_cont),
            ("cat",  cat_pipe,  present_cat),
            ("bin",  bin_pipe,  present_bin),
        ],
        remainder="drop",
        verbose_feature_names_out=False
    )

preprocessor_dense = make_dense_preprocessor()

# ---------- Models ----------
models = {
    "lr_elasticnet": Pipeline([
        ("prep", preprocessor_sparse),
        ("clf", LogisticRegression(
            penalty="elasticnet",
            l1_ratio=0.2,        # light L1 for stability; can tune 0.1–0.5
            solver="saga",       # supports elastic net + sparse
            C=1.0,
            max_iter=5000,
            n_jobs=-1,
            random_state=42
        ))
    ]),
    "random_forest": Pipeline([
        ("prep", preprocessor_sparse),
        ("clf", RandomForestClassifier(
            n_estimators=500,
            max_depth=None,
            min_samples_split=4,
            min_samples_leaf=2,
            max_features="sqrt",
            n_jobs=-1,
            random_state=42
        ))
    ]),
    "hist_gb": Pipeline([
        ("prep", preprocessor_dense),  # dense
        ("clf", HistGradientBoostingClassifier(
            loss="log_loss",
            learning_rate=0.08,
            max_depth=None,
            max_bins=255,
            l2_regularization=0.0,
            early_stopping=True,
            random_state=42
        ))
    ]),
}

# ---------- CV + OOF evaluation ----------
Path("artifacts").mkdir(exist_ok=True)
n_splits = 5
cv = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state=42)

all_oof = {}
summary_rows = []

for name, pipe in models.items():
    print(f"\n=== {name} ===")
    oof_pred = np.zeros(len(y_all), dtype=float)
    fold_metrics = []

    for fold, (tr_idx, va_idx) in enumerate(cv.split(X_all, y_all, groups=groups), start=1):
        X_tr, X_va = X_all.iloc[tr_idx], X_all.iloc[va_idx]
        y_tr, y_va = y_all.iloc[tr_idx], y_all.iloc[va_idx]

        model = pipe
        model.fit(X_tr, y_tr)

        p = model.predict_proba(X_va)[:, 1]
        oof_pred[va_idx] = p

        auc  = roc_auc_score(y_va, p)
        pr   = average_precision_score(y_va, p)
        bri  = brier_score_loss(y_va, p)
        fold_metrics.append((auc, pr, bri))
        print(f"[Fold {fold}] ROC-AUC={auc:.3f}  PR-AUC={pr:.3f}  Brier={bri:.3f}")

    # Overall OOF
    auc_oof = roc_auc_score(y_all, oof_pred)
    pr_oof  = average_precision_score(y_all, oof_pred)
    bri_oof = brier_score_loss(y_all, oof_pred)

    fm = np.array(fold_metrics)
    print("\nOOF:")
    print(f"ROC-AUC={auc_oof:.3f}  PR-AUC={pr_oof:.3f}  Brier={bri_oof:.3f}")
    print("Per-fold means ± SD:")
    print(f"AUC  : {fm[:,0].mean():.3f} ± {fm[:,0].std():.3f}")
    print(f"PR   : {fm[:,1].mean():.3f} ± {fm[:,1].std():.3f}")
    print(f"Brier: {fm[:,2].mean():.3f} ± {fm[:,2].std():.3f}")

    all_oof[name] = oof_pred
    summary_rows.append({
        "model": name,
        "auc_oof": auc_oof,
        "prauc_oof": pr_oof,
        "brier_oof": bri_oof,
        "auc_mean": fm[:,0].mean(), "auc_sd": fm[:,0].std(),
        "pr_mean": fm[:,1].mean(),  "pr_sd": fm[:,1].std(),
        "brier_mean": fm[:,2].mean(), "brier_sd": fm[:,2].std(),
    })

# ---------- Save artifacts ----------
oof_df = pd.DataFrame(all_oof, index=df_mod.index)
oof_df.to_parquet("artifacts/oof_baselines.parquet", index=True)

summary_df = pd.DataFrame(summary_rows).sort_values("auc_oof", ascending=False)
summary_df.to_csv("artifacts/baseline_summary.csv", index=False)

print("\nSaved → artifacts/oof_baselines.parquet")
print("Saved → artifacts/baseline_summary.csv")
print("\nBaseline OOF summary (sorted by AUC):")
display(summary_df)


/home/jupyter-shikshuk/.local/lib/python3.9/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:19: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(



=== lr_elasticnet ===
[Fold 1] ROC-AUC=0.824  PR-AUC=0.802  Brier=0.173
[Fold 2] ROC-AUC=0.806  PR-AUC=0.791  Brier=0.181
[Fold 3] ROC-AUC=0.806  PR-AUC=0.784  Brier=0.181
[Fold 4] ROC-AUC=0.820  PR-AUC=0.822  Brier=0.176
[Fold 5] ROC-AUC=0.798  PR-AUC=0.777  Brier=0.184

OOF:
ROC-AUC=0.809  PR-AUC=0.792  Brier=0.179
Per-fold means ± SD:
AUC  : 0.811 ± 0.010
PR   : 0.795 ± 0.016
Brier: 0.179 ± 0.004

=== random_forest ===
[Fold 1] ROC-AUC=0.832  PR-AUC=0.810  Brier=0.171
[Fold 2] ROC-AUC=0.821  PR-AUC=0.815  Brier=0.176
[Fold 3] ROC-AUC=0.816  PR-AUC=0.792  Brier=0.177
[Fold 4] ROC-AUC=0.831  PR-AUC=0.839  Brier=0.172
[Fold 5] ROC-AUC=0.810  PR-AUC=0.795  Brier=0.179

OOF:
ROC-AUC=0.822  PR-AUC=0.809  Brier=0.175
Per-fold means ± SD:
AUC  : 0.822 ± 0.008
PR   : 0.810 ± 0.017
Brier: 0.175 ± 0.003

=== hist_gb ===
[Fold 1] ROC-AUC=0.859  PR-AUC=0.835  Brier=0.152
[Fold 2] ROC-AUC=0.847  PR-AUC=0.835  Brier=0.159
[Fold 3] ROC-AUC=0.843  PR-AUC=0.813  Brier=0.159
[Fold 4] ROC-AUC=0.869  P

,model,auc_oof,prauc_oof,brier_oof,auc_mean,auc_sd,pr_mean,pr_sd,brier_mean,brier_sd
2,hist_gb,0.851273,0.835078,0.156635,0.851026,0.011523,0.836309,0.020761,0.156635,0.005911
1,random_forest,0.821932,0.809246,0.175141,0.822027,0.008290,0.810209,0.016750,0.175140,0.002855
0,lr_elasticnet,0.809280,0.792245,0.178775,0.810504,0.009604,0.795111,0.015837,0.178775,0.004088


## Step C1: XGBoost with the same preprocessor and patient-level grouped CV, using light, nested tuning (no leakage).

In [24]:
# C1) XGBoost with patient-grouped nested CV (light tuning) → OOF metrics

import json
import numpy as np
import pandas as pd
import joblib
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedGroupKFold, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss
from xgboost import XGBClassifier

# 0) Load artifacts from Step A
preprocessor = joblib.load("artifacts/preprocessor_unfitted.joblib")
with open("artifacts/preprocessor_spec.json", "r") as f:
    spec = json.load(f)

TARGET   = spec["target"]
EXCLUDED = spec["excluded"]

# 1) Build X/y; set patient groups (STUDY_ID)
drop_cols = list(set(EXCLUDED + [TARGET]))
X_all = df_mod.drop(columns=drop_cols).copy()
y_all = df_mod[TARGET].astype(int).copy()

if "study_id" not in df_mod.columns:
    raise ValueError("study_id column is required for patient-level grouped CV.")
groups_all = df_mod["study_id"].values

# 2) Base pipeline (preprocessor + XGB); parameters to be tuned lightly
xgb = XGBClassifier(
    objective="binary:logistic",
    eval_metric="auc",
    tree_method="hist",
    n_estimators=800,       # use enough trees; learning_rate tuned below
    random_state=42
)

pipe = Pipeline([
    ("prep", preprocessor),
    ("clf",  xgb),
])

# 3) Light search space (compact but effective)
param_dist = {
    "clf__max_depth":        [3, 4, 5, 6],
    "clf__learning_rate":    np.linspace(0.01, 0.06, 6),
    "clf__subsample":        np.linspace(0.6, 0.95, 8),
    "clf__colsample_bytree": np.linspace(0.6, 0.95, 8),
    "clf__min_child_weight": [1.0, 3.0, 5.0, 7.0],
    "clf__reg_lambda":       [0.0, 0.5, 1.0, 2.0, 5.0],
    "clf__reg_alpha":        [0.0, 0.1, 0.5, 1.0],
    "clf__gamma":            [0.0, 0.1, 0.5, 1.0],
    "clf__max_delta_step":   [0, 1, 2],
}

# 4) Outer CV (patient-grouped); inner CV for tuning on train folds only
outer_cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)

oof_pred = np.zeros(len(y_all), dtype=float)
fold_metrics = []
fold_params  = []

for fold, (tr_idx, va_idx) in enumerate(outer_cv.split(X_all, y_all, groups=groups_all), start=1):
    X_tr, X_va = X_all.iloc[tr_idx], X_all.iloc[va_idx]
    y_tr, y_va = y_all.iloc[tr_idx], y_all.iloc[va_idx]
    groups_tr  = groups_all[tr_idx]

    # Inner CV for tuning (grouped by patient)
    inner_cv = StratifiedGroupKFold(n_splits=4, shuffle=True, random_state=43 + fold)

    search = RandomizedSearchCV(
        estimator=pipe,
        param_distributions=param_dist,
        n_iter=40,                 # light tuning
        scoring="roc_auc",
        cv=inner_cv.split(X_tr, y_tr, groups=groups_tr),
        n_jobs=-1,
        refit=True,
        random_state=123 + fold,
        verbose=0
    )

    search.fit(X_tr, y_tr)
    best_pipe = search.best_estimator_
    fold_params.append(search.best_params_)

    # Predict on the held-out outer fold
    p = best_pipe.predict_proba(X_va)[:, 1]
    oof_pred[va_idx] = p

    auc  = roc_auc_score(y_va, p)
    pr   = average_precision_score(y_va, p)
    bri  = brier_score_loss(y_va, p)
    fold_metrics.append((auc, pr, bri))
    print(f"[Outer Fold {fold}] ROC-AUC={auc:.3f}  PR-AUC={pr:.3f}  Brier={bri:.3f}")
    print(f"  Best params: {search.best_params_}")

# 5) Overall OOF performance
auc_oof = roc_auc_score(y_all, oof_pred)
pr_oof  = average_precision_score(y_all, oof_pred)
bri_oof = brier_score_loss(y_all, oof_pred)

print("\n=== Cross-validated (OOF) performance — XGBoost (light tuned) ===")
print(f"ROC-AUC={auc_oof:.3f}  PR-AUC={pr_oof:.3f}  Brier={bri_oof:.3f}")

# 6) Per-fold summary
fold_metrics = np.array(fold_metrics)
print("\nPer-fold means ± SD:")
print(f"AUC  : {fold_metrics[:,0].mean():.3f} ± {fold_metrics[:,0].std():.3f}")
print(f"PR   : {fold_metrics[:,1].mean():.3f} ± {fold_metrics[:,1].std():.3f}")
print(f"Brier: {fold_metrics[:,2].mean():.3f} ± {fold_metrics[:,2].std():.3f}")


[Outer Fold 1] ROC-AUC=0.838  PR-AUC=0.815  Brier=0.165
  Best params: {'clf__subsample': np.float64(0.7), 'clf__reg_lambda': 1.0, 'clf__reg_alpha': 0.0, 'clf__min_child_weight': 1.0, 'clf__max_depth': 4, 'clf__max_delta_step': 2, 'clf__learning_rate': np.float64(0.01), 'clf__gamma': 0.5, 'clf__colsample_bytree': np.float64(0.6)}
[Outer Fold 2] ROC-AUC=0.829  PR-AUC=0.823  Brier=0.168
  Best params: {'clf__subsample': np.float64(0.65), 'clf__reg_lambda': 0.0, 'clf__reg_alpha': 0.5, 'clf__min_child_weight': 1.0, 'clf__max_depth': 6, 'clf__max_delta_step': 0, 'clf__learning_rate': np.float64(0.01), 'clf__gamma': 0.1, 'clf__colsample_bytree': np.float64(0.85)}
[Outer Fold 3] ROC-AUC=0.820  PR-AUC=0.792  Brier=0.172
  Best params: {'clf__subsample': np.float64(0.65), 'clf__reg_lambda': 1.0, 'clf__reg_alpha': 0.0, 'clf__min_child_weight': 1.0, 'clf__max_depth': 5, 'clf__max_delta_step': 2, 'clf__learning_rate': np.float64(0.01), 'clf__gamma': 0.5, 'clf__colsample_bytree': np.float64(0.7)}
[

# Step C2 - XGBoost Hyperparameter Tuning + Isotonic Calibration with Patient-Grouped Cross-Validation

In [25]:
# C2) XGBoost (expanded tuning + isotonic calibration) with patient-grouped CV
import json, numpy as np, pandas as pd, joblib
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedGroupKFold, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss
from sklearn.calibration import CalibratedClassifierCV
from xgboost import XGBClassifier

# 0) Load artifacts
preprocessor = joblib.load("artifacts/preprocessor_unfitted.joblib")
with open("artifacts/preprocessor_spec.json", "r") as f:
    spec = json.load(f)

TARGET, EXCLUDED = spec["target"], spec["excluded"]
drop_cols = list(set(EXCLUDED + [TARGET]))
X_all = df_mod.drop(columns=drop_cols).copy()
y_all = df_mod[TARGET].astype(int).copy()
groups_all = df_mod["study_id"].values

# 1) Base model
xgb = XGBClassifier(
    objective="binary:logistic",
    eval_metric="auc",
    tree_method="hist",
    n_estimators=1200,
    random_state=42
)
pipe = Pipeline([
    ("prep", preprocessor),
    ("clf",  xgb),
])

# 2) Expanded search space
param_dist = {
    "clf__max_depth":        [3, 4, 5, 6, 7],
    "clf__learning_rate":    np.linspace(0.005, 0.05, 6),
    "clf__subsample":        np.linspace(0.6, 1.0, 9),
    "clf__colsample_bytree": np.linspace(0.6, 1.0, 9),
    "clf__min_child_weight": [1, 3, 5, 7],
    "clf__gamma":            [0.0, 0.1, 0.3, 0.5],
    "clf__reg_lambda":       [0.0, 0.5, 1.0, 2.0, 5.0],
    "clf__reg_alpha":        [0.0, 0.1, 0.5, 1.0],
}

# 3) Outer CV
outer_cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)
oof_pred, fold_metrics = np.zeros(len(y_all)), []

for fold, (tr_idx, va_idx) in enumerate(outer_cv.split(X_all, y_all, groups=groups_all), start=1):
    X_tr, X_va = X_all.iloc[tr_idx], X_all.iloc[va_idx]
    y_tr, y_va = y_all.iloc[tr_idx], y_all.iloc[va_idx]
    groups_tr  = groups_all[tr_idx]

    # Inner tuning
    inner_cv = StratifiedGroupKFold(n_splits=4, shuffle=True, random_state=43 + fold)
    search = RandomizedSearchCV(
        estimator=pipe,
        param_distributions=param_dist,
        n_iter=60,
        scoring="roc_auc",
        cv=inner_cv.split(X_tr, y_tr, groups=groups_tr),
        n_jobs=-1,
        refit=True,
        random_state=123 + fold,
        verbose=0
    )
    search.fit(X_tr, y_tr)
    best_pipe = search.best_estimator_

    # --- Calibration on validation fold ---
    cal = CalibratedClassifierCV(best_pipe.named_steps["clf"], cv="prefit", method="isotonic")
    cal.fit(best_pipe.named_steps["prep"].transform(X_va), y_va)

    # Predict
    p = cal.predict_proba(best_pipe.named_steps["prep"].transform(X_va))[:, 1]
    oof_pred[va_idx] = p

    auc, pr, bri = roc_auc_score(y_va, p), average_precision_score(y_va, p), brier_score_loss(y_va, p)
    fold_metrics.append((auc, pr, bri))
    print(f"[Fold {fold}] ROC-AUC={auc:.3f}  PR-AUC={pr:.3f}  Brier={bri:.3f}")
    print(f"  Best params: {search.best_params_}")

# 4) Overall metrics
auc_oof = roc_auc_score(y_all, oof_pred)
pr_oof  = average_precision_score(y_all, oof_pred)
bri_oof = brier_score_loss(y_all, oof_pred)

print("\n=== Cross-validated (OOF) — XGBoost expanded + calibrated ===")
print(f"ROC-AUC={auc_oof:.3f}  PR-AUC={pr_oof:.3f}  Brier={bri_oof:.3f}")

fold_metrics = np.array(fold_metrics)
print("\nPer-fold means ± SD:")
print(f"AUC  : {fold_metrics[:,0].mean():.3f} ± {fold_metrics[:,0].std():.3f}")
print(f"PR   : {fold_metrics[:,1].mean():.3f} ± {fold_metrics[:,1].std():.3f}")
print(f"Brier: {fold_metrics[:,2].mean():.3f} ± {fold_metrics[:,2].std():.3f}")


/home/jupyter-shikshuk/.local/lib/python3.9/site-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(


[Fold 1] ROC-AUC=0.844  PR-AUC=0.812  Brier=0.160
  Best params: {'clf__subsample': np.float64(0.8), 'clf__reg_lambda': 5.0, 'clf__reg_alpha': 0.1, 'clf__min_child_weight': 3, 'clf__max_depth': 6, 'clf__learning_rate': np.float64(0.005), 'clf__gamma': 0.3, 'clf__colsample_bytree': np.float64(0.65)}


/home/jupyter-shikshuk/.local/lib/python3.9/site-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(


[Fold 2] ROC-AUC=0.836  PR-AUC=0.810  Brier=0.163
  Best params: {'clf__subsample': np.float64(0.65), 'clf__reg_lambda': 2.0, 'clf__reg_alpha': 0.5, 'clf__min_child_weight': 3, 'clf__max_depth': 5, 'clf__learning_rate': np.float64(0.014000000000000002), 'clf__gamma': 0.1, 'clf__colsample_bytree': np.float64(0.65)}


/home/jupyter-shikshuk/.local/lib/python3.9/site-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(


[Fold 3] ROC-AUC=0.823  PR-AUC=0.786  Brier=0.168
  Best params: {'clf__subsample': np.float64(0.6), 'clf__reg_lambda': 0.0, 'clf__reg_alpha': 0.5, 'clf__min_child_weight': 1, 'clf__max_depth': 6, 'clf__learning_rate': np.float64(0.005), 'clf__gamma': 0.1, 'clf__colsample_bytree': np.float64(0.85)}


/home/jupyter-shikshuk/.local/lib/python3.9/site-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(


[Fold 4] ROC-AUC=0.843  PR-AUC=0.834  Brier=0.159
  Best params: {'clf__subsample': np.float64(0.8), 'clf__reg_lambda': 5.0, 'clf__reg_alpha': 0.0, 'clf__min_child_weight': 3, 'clf__max_depth': 5, 'clf__learning_rate': np.float64(0.014000000000000002), 'clf__gamma': 0.0, 'clf__colsample_bytree': np.float64(0.6)}
[Fold 5] ROC-AUC=0.817  PR-AUC=0.791  Brier=0.173
  Best params: {'clf__subsample': np.float64(0.75), 'clf__reg_lambda': 5.0, 'clf__reg_alpha': 0.0, 'clf__min_child_weight': 1, 'clf__max_depth': 3, 'clf__learning_rate': np.float64(0.014000000000000002), 'clf__gamma': 0.1, 'clf__colsample_bytree': np.float64(0.85)}

=== Cross-validated (OOF) — XGBoost expanded + calibrated ===
ROC-AUC=0.836  PR-AUC=0.826  Brier=0.164

Per-fold means ± SD:
AUC  : 0.832 ± 0.011
PR   : 0.807 ± 0.017
Brier: 0.164 ± 0.005


/home/jupyter-shikshuk/.local/lib/python3.9/site-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(


In [ ]:
# End of Notebook